In [1]:
import pandas as pd
import random
import json
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
import joblib
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from tqdm import tqdm
import string
import scipy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import f1_score
from IPython.display import HTML, display
from tqdm import tqdm
import spacy
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordTokenizer
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
tokenizer = TreebankWordTokenizer()

[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/jimbo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/jimbo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jimbo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
data_0 = pd.read_csv('../datasets/fossology-master-corrected.csv')
X_0 = data_0["copyright"]
y_0 = data_0["falsePositive"]
X_0 = X_0.drop_duplicates()
y_0 = y_0[X_0.index]

data_1 = pd.read_csv('../datasets/kubernetes-master-corrected.csv')
X_1 = data_1["copyright"]
y_1 = data_1["falsePositive"]
X_1 = X_1.drop_duplicates()
y_1 = y_1[X_1.index]

data_2 = pd.read_csv('../datasets/tensorflow-master-corrected.csv')
X_2 = data_2["copyright"]
y_2 = data_2["falsePositive"]
X_2 = X_2.drop_duplicates()
y_2 = y_2[X_2.index]

data_3 = pd.read_csv('../datasets/fossology-provided-1-corrected.csv')

X_3 = data_3['copyright']
y_3 = data_3['falsePositive']
X_3 = X_3.drop_duplicates()
y_3 = y_3[X_3.index]

data_4 = pd.read_csv('../datasets/fossology-provided-2.csv')

X_4 = data_4['copyright']
y_4 = data_4['falsePositive']
X_4 = X_4.drop_duplicates()
y_4 = y_4[X_4.index]

data_5 = pd.read_csv('../datasets/feature-extraction-paper.csv')

X_5 = data_5['copyright']
y_5 = data_5['falsePositive']
X_5 = X_5.drop_duplicates()
y_5 = y_5[X_5.index]

X = pd.concat([X_0, X_1, X_2, X_3])
y = pd.concat([y_0, y_1, y_2, y_3])

X = X.reset_index(drop=True)
y = y.reset_index(drop=True)

print('Class 0 Percentage: ', len(y[y == 0]) / len(y))
print('Class 1 Percentage: ', len(y[y == 1]) / len(y))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

Class 0 Percentage:  0.7522737712448323
Class 1 Percentage:  0.24772622875516767


In [3]:
def aggregate_reports(reports, print_aggregates=True):
    import pandas as pd
    import numpy as np
    dfs = []
    for metric in ['precision', 'recall', 'f1-score']:
        scores = []
        for report in reports:
            scores.append([report['0'][metric], report['1'][metric]])
        scores = np.array(scores)
        scores = scores[:, :2]
        mean_scores = np.mean(scores, axis=0)
        mean_scores = [f"{score:.6f}" for score in mean_scores]
        df = pd.DataFrame(scores, columns=['0', '1'])
        df.loc['Mean'] = mean_scores
        df['Metric'] = metric
        dfs.append(df)
    if print_aggregates:
        print("## Precision")
        print(dfs[0].to_markdown())
        print("## Recall")
        print(dfs[1].to_markdown())
        print("## F1-score")
        print(dfs[2].to_markdown())
    else:
        return dfs[0], dfs[1], dfs[2]

def get_missclassified_rows(X, y_true, y_pred, only_this_class = [0, 1], return_index=False):
    if type(y_true) != list:
        y_true = y_true.tolist()
    if type(y_pred) != list:
        y_pred = y_pred.tolist()
    if type(X) != list:
        X = X.tolist()
    missclassified_rows = []
    for i in range(len(y_true)):
        if y_true[i] != y_pred[i] and y_true[i] in only_this_class:
            missclassified_rows.append(i)
    if return_index:
        return [(y_pred[i], i, X[i]) for i in missclassified_rows]
    else:
        return [(y_pred[i], X[i]) for i in missclassified_rows]

def preprocess_function(sentences, lower=False, replace_copyright_symbols=False,
                        replace_dates=False, remove_numbers=False, remove_punctuation=False,
                        remove_special_characters=False, remove_whitespaces=False, remove_specials_weird=False,
                        remove_stopwords=False, replace_emails=False,
                        tokenize=False, lemmatize=False, glove=False, remove_endlines=False,
                        replace_copyright_symbol_v2=False, replace_entities=False, spacy_model=None):
    if type(sentences) is not list:
        sentences = sentences.to_list()
    for i in range(len(sentences)):
        sentences[i] = str(sentences[i])
    if replace_entities:
        if spacy_model is None: # give error
            raise ValueError('spacy_model must be provided if replace_entities is True')
        else:
            nlp = spacy_model
        sentences = [nlp(sentence) for sentence in sentences]
        new_sentences = []
        for sentence in sentences:
            new_sentence = sentence.text
            for entity in sentence.ents:
                if entity.label_ == 'ENT':
                    new_sentence = re.sub(re.escape(entity.text), ' ENTITY ', new_sentence)
            new_sentences.append(new_sentence)
        sentences = new_sentences
    if replace_dates:
        if glove:
            sentences = [re.sub(r'\d{4}', ' <DATE> ', sentence) for sentence in sentences]
        else:
            sentences = [re.sub(r'\d{4}', ' DATE ', sentence) for sentence in sentences]
    if remove_numbers:
        sentences = [re.sub(r'\d+', ' ', sentence) for sentence in sentences]
    if replace_copyright_symbols:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHTSYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_copyright_symbol_v2:
        if glove:
            symbol_text = ' <COPYRIGHT SYMBOL> '
        else:
            symbol_text = ' COPYRIGHT SYMBOL '
        sentences = [re.sub(r'©', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(c\)', symbol_text, sentence) for sentence in sentences]
        sentences = [re.sub(r'\(C\)', symbol_text, sentence) for sentence in sentences]
    if replace_emails:
        if glove:
            email_text = ' <EMAIL> '
        else:
            email_text = ' EMAIL '
        sentences = [re.sub("""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)*|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9]))\.){3}(?:(2(5[0-5]|[0-4][0-9])|1[0-9][0-9]|[1-9]?[0-9])|[a-z0-9-]*[a-z0-9]:(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])""", email_text, sentence) for sentence in sentences]
    if tokenize:
        if not glove:
            sentences = [''.join(tokenizer.tokenize(sentence)) for sentence in sentences]
        else:
            sentences = [tokenizer.tokenize(sentence) for sentence in sentences]
    if lemmatize:
        lemmatizer = WordNetLemmatizer()
        if not glove:
            sentences = [''.join([lemmatizer.lemmatize(word) for word in sentence]) for sentence in sentences]
        else:
            sentences = [[lemmatizer.lemmatize(word) for word in sentence] for sentence in sentences]
    if remove_punctuation:
        sentences = [re.sub(r'[^\w\s]', ' ', sentence) for sentence in sentences]
    if remove_special_characters:
        sentences = [re.sub(r'[^a-zA-Z0-9]', ' ', sentence) for sentence in sentences]
    if remove_specials_weird:
        sentences = [re.sub(r'[^a-zA-Z0-9]', '', sentence) for sentence in sentences]
    if lower:
        sentences = [sentence.lower() for sentence in sentences]
    if remove_stopwords:
        sentences = [re.sub(r'\b(?:{})\b'.format('|'.join(stopwords.words('english'))), ' ', sentence)
                     for sentence in sentences]
    if remove_endlines:
        sentences = [re.sub(r'\n', ' ', sentence) for sentence in sentences]
    if remove_whitespaces:
        sentences = [re.sub(r' {2,}', ' ', sentence) for sentence in sentences]
    return sentences

def train(svm, vectorizer, threshold, preprocess_function,**kwargs):
    X_train_tfidf = vectorizer.fit_transform(preprocess_function(X_train, **kwargs))
    X_test_tfidf = vectorizer.transform(preprocess_function(X_test, **kwargs))
    X_1_tfidf = vectorizer.transform(preprocess_function(X_1, **kwargs))
    X_2_tfidf = vectorizer.transform(preprocess_function(X_2, **kwargs))
    X_3_tfidf = vectorizer.transform(preprocess_function(X_3, **kwargs))
    preprocessed_X = preprocess_function(X, **kwargs)
    X_tfidf = vectorizer.transform(preprocessed_X)
    svm.fit(X_train_tfidf, y_train)
    if True: #svm.probability:
        y_pred = svm.predict_proba(X_test_tfidf)
        y_pred_1 = svm.predict_proba(X_1_tfidf)
        y_pred_2 = svm.predict_proba(X_2_tfidf)
        y_pred_3 = svm.predict_proba(X_3_tfidf)
        y_pred_4 = svm.predict_proba(X_tfidf)
        if threshold is None:
            y_pred_classification = np.argmax(y_pred, axis=1)
            y_pred_1_classification = np.argmax(y_pred_1, axis=1)
            y_pred_2_classification = np.argmax(y_pred_2, axis=1)
            y_pred_3_classification = np.argmax(y_pred_3, axis=1)
            y_pred_4_classification = np.argmax(y_pred_4, axis=1)
        else:
            y_pred_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred]
            y_pred_1_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_1]
            y_pred_2_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_2]
            y_pred_3_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_3]
            y_pred_4_classification = [np.argmax(y) if max(y) > threshold else 0 for y in y_pred_4]
    else:
        y_pred_classification = svm.predict(X_test_tfidf)
        y_pred_1_classification = svm.predict(X_1_tfidf)
        y_pred_2_classification = svm.predict(X_2_tfidf)
        y_pred_3_classification = svm.predict(X_3_tfidf)
        y_pred_4_classification = svm.predict(X_tfidf)
    report = classification_report(y_test, y_pred_classification, output_dict=True)
    report_1 = classification_report(y_1, y_pred_1_classification, output_dict=True)
    report_2 = classification_report(y_2, y_pred_2_classification, output_dict=True)
    report_3 = classification_report(y_3, y_pred_3_classification, output_dict=True)
    report_4 = classification_report(y, y_pred_4_classification, output_dict=True)
    miss_classified_rows_0 = get_missclassified_rows(preprocessed_X, y, y_pred_4_classification, only_this_class=[0], return_index=True)
    miss_classified_rows_1 = get_missclassified_rows(preprocessed_X, y, y_pred_4_classification, only_this_class=[1], return_index=True)
    #aggregate_reports([report, report_1, report_2, report_3, report_4])
    print('Number of missclassifications in class 0: ', report_4['0']['support'] - round(report_4['0']['recall'] * report_4['0']['support']), 'out of a total sample of: ', report_4['0']['support'], ' - about ', round((1 - report_4['0']['recall']) * 100, 2), '% of the class was missclassified')
    print('Number of missclassifications in class 1: ', report_4['1']['support'] - round(report_4['1']['recall'] * report_4['1']['support']), 'out of a total sample of: ', report_4['1']['support'], ' - about ', round((1 - report_4['1']['recall']) * 100, 2), '% of the class was missclassified')
    return svm, vectorizer, miss_classified_rows_0, miss_classified_rows_1, preprocessed_X

In [5]:
def write_model_outputs_to_disk(X, model_path, path=None):
    model = spacy.load(model_path)
    X_enitities = [model(sentence) for sentence in tqdm(X)]
    train_docs, valid_docs = train_test_split(X_enitities, test_size=0.2, random_state=42, shuffle=True)
    train_doc_bin = spacy.tokens.DocBin(docs=train_docs)
    valid_doc_bin = spacy.tokens.DocBin(docs=valid_docs)
    if path is None:
        files = os.listdir('../NER_datasets/spacy_compatible/copyright-data/')
        files = [file for file in files if file.startswith('entire-dataset')]
        iteration_number = int(len(files) / 2 + 1)
        train_path = '../NER_datasets/spacy_compatible/copyright-data/entire-dataset-semi-supervised-train-' \
                    + str(iteration_number) + '.spacy'
        valid_path = '../NER_datasets/spacy_compatible/copyright-data/entire-dataset-semi-supervised-valid-' \
                    + str(iteration_number) + '.spacy'
    else:
        train_path = path + '-train.spacy'
        valid_path = path + '-valid.spacy'
    train_doc_bin.to_disk(train_path)
    valid_doc_bin.to_disk(valid_path)

def create_semi_supervised_config_file():
    files = os.listdir('../NER_configs/')
    files = [file for file in files if file.startswith('train-semi-supervised')]
    iteration_number = len(files) + 1
    
    with open(os.path.join('../NER_configs/', files[-1]), 'r', encoding='utf-8') as file:
        cfg_contents = file.read()
    
    new_train_path = f'../NER_datasets/spacy_compatible/copyright-data/entire-dataset-semi-supervised-train-{iteration_number}.spacy'
    new_dev_path = f'../NER_datasets/spacy_compatible/copyright-data/entire-dataset-semi-supervised-valid-{iteration_number}.spacy'
    
    cfg_contents = re.sub(r'train\s*=\s*".*"', f'train = "{new_train_path}"', cfg_contents)
    cfg_contents = re.sub(r'dev\s*=\s*".*"', f'dev = "{new_dev_path}"', cfg_contents)
    
    new_cfg_file_path = f'../NER_configs/train-semi-supervised-dataset-{iteration_number}.cfg'
    with open(new_cfg_file_path, 'w', encoding='utf-8') as file:
        file.write(cfg_contents)z

def train_semi_supervised_ner_model(X, model_path=None, path=None, epochs=10):
    for i in range(epochs):
        if model_path is None:
            files = os.listdir('../NER_models/')
            files = [file for file in files if file.startswith('train-semi-supervised-dataset')]
            iteration_number = len(files)
            model_path = f'../NER_models/train-semi-supervised-dataset-{iteration_number}/model-best'
        write_model_outputs_to_disk(X, model_path, path)
        create_semi_supervised_config_file()

        files = os.listdir('../NER_models/')
        files = [file for file in files if file.startswith('train-semi-supervised-dataset')]
        iteration_number = len(files) + 1
        print('Training model number: ', iteration_number)
        train_command = f"python -m spacy train '../NER_configs/train-semi-supervised-dataset-{iteration_number}.cfg' --output '../NER_models/train-semi-supervised-dataset-{iteration_number}'"
        os.system(train_command)   

In [ ]:
# In case I need to go through more iterations (currently not as F1-Score is 0.965)
train_semi_supervised_ner_model(X, epochs=3)

In [10]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
test = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), None, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/train-semi-supervised-dataset-1/model-best'))

Number of missclassifications in class 0:  25.0 out of a total sample of:  16377.0  - about  0.15 % of the class was missclassified
Number of missclassifications in class 1:  43.0 out of a total sample of:  5393.0  - about  0.8 % of the class was missclassified


In [6]:
clf = OneVsRestClassifier(SVC(probability=True, C=25))
best_model = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), 0.99, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/train-semi-supervised-dataset-1/model-best'))

Number of missclassifications in class 0:  2.0 out of a total sample of:  16377.0  - about  0.01 % of the class was missclassified
Number of missclassifications in class 1:  318.0 out of a total sample of:  5393.0  - about  5.9 % of the class was missclassified


In [11]:
clf = SVC(probability=True, C=25)
test_2 = train(clf, TfidfVectorizer(ngram_range=(1, 2), binary=True), 0.99, preprocess_function,
                                            replace_dates=True, remove_numbers=True, replace_copyright_symbols=True,
                                            remove_whitespaces=True, lower=True, remove_special_characters=True, 
                                            replace_emails=True, replace_entities=True,
                                            spacy_model=spacy.load('../NER_models/train-semi-supervised-dataset-1/model-best'))

Number of missclassifications in class 0:  3.0 out of a total sample of:  16377.0  - about  0.02 % of the class was missclassified
Number of missclassifications in class 1:  301.0 out of a total sample of:  5393.0  - about  5.58 % of the class was missclassified


In [5]:
from copyrightfpd.CopyrightFPD import *

In [ ]:
c = CopyrightFPD()
test_predictions = c.predict(X_test, 0.5)

In [13]:
#test_predictions = [0 if prediction == 't' else 1 for prediction in test_predictions]
print(aggregate_reports([classification_report(y_test, test_predictions, output_dict=True)]))

## Precision
|      |        0 |        1 | Metric    |
|:-----|---------:|---------:|:----------|
| 0    | 0.992336 | 0.978022 | precision |
| Mean | 0.992336 | 0.978022 | precision |
## Recall
|      |       0 |        1 | Metric   |
|:-----|--------:|---------:|:---------|
| 0    | 0.99264 | 0.977127 | recall   |
| Mean | 0.99264 | 0.977127 | recall   |
## F1-score
|      |        0 |        1 | Metric   |
|:-----|---------:|---------:|:---------|
| 0    | 0.992488 | 0.977574 | f1-score |
| Mean | 0.992488 | 0.977574 | f1-score |
None


In [14]:
classification_report(y_test, test_predictions, output_dict=True)

{'0': {'precision': 0.9923359901900675,
  'recall': 0.9926402943882244,
  'f1-score': 0.9924881189636671,
  'support': 3261.0},
 '1': {'precision': 0.978021978021978,
  'recall': 0.9771271729185728,
  'f1-score': 0.9775743707093821,
  'support': 1093.0},
 'accuracy': 0.9887459807073955,
 'macro avg': {'precision': 0.9851789841060228,
  'recall': 0.9848837336533987,
  'f1-score': 0.9850312448365246,
  'support': 4354.0},
 'weighted avg': {'precision': 0.9887426931529242,
  'recall': 0.9887459807073955,
  'f1-score': 0.9887442680583078,
  'support': 4354.0}}

In [15]:
test_predictions_1 = c.predict(X_1, 0.5)
test_predictions_1 = [0 if prediction == 't' else 1 for prediction in test_predictions_1]
classification_report(y_1, test_predictions_1, output_dict=True)

{'0': {'precision': 0.9956709956709957,
  'recall': 0.9871244635193133,
  'f1-score': 0.9913793103448276,
  'support': 466.0},
 '1': {'precision': 0.9478260869565217,
  'recall': 0.9819819819819819,
  'f1-score': 0.9646017699115044,
  'support': 111.0},
 'accuracy': 0.9861351819757366,
 'macro avg': {'precision': 0.9717485413137588,
  'recall': 0.9845532227506476,
  'f1-score': 0.977990540128166,
  'support': 577.0},
 'weighted avg': {'precision': 0.9864668624520936,
  'recall': 0.9861351819757366,
  'f1-score': 0.9862279984070479,
  'support': 577.0}}

In [16]:
test_predictions_2 = c.predict(X_2, 0.5)
test_predictions_2 = [0 if prediction == 't' else 1 for prediction in test_predictions_2]
classification_report(y_2, test_predictions_2, output_dict=True)

{'0': {'precision': 0.9738562091503268,
  'recall': 0.9612903225806452,
  'f1-score': 0.9675324675324675,
  'support': 155.0},
 '1': {'precision': 0.9375,
  'recall': 0.9574468085106383,
  'f1-score': 0.9473684210526315,
  'support': 94.0},
 'accuracy': 0.9598393574297188,
 'macro avg': {'precision': 0.9556781045751634,
  'recall': 0.9593685655456418,
  'f1-score': 0.9574504442925496,
  'support': 249.0},
 'weighted avg': {'precision': 0.9601313751738982,
  'recall': 0.9598393574297188,
  'f1-score': 0.9599203375360635,
  'support': 249.0}}

In [17]:
test_predictions_5 = c.predict(X_5, 0.5)
test_predictions_5 = [0 if prediction == 't' else 1 for prediction in test_predictions_5]
classification_report(y_5, test_predictions_5, output_dict=True)

{'0': {'precision': 0.9994939271255061,
  'recall': 0.9989883662114315,
  'f1-score': 0.9992410827219833,
  'support': 1977.0},
 '1': {'precision': 0.9809523809523809,
  'recall': 0.9903846153846154,
  'f1-score': 0.985645933014354,
  'support': 104.0},
 'accuracy': 0.9985583853916387,
 'macro avg': {'precision': 0.9902231540389435,
  'recall': 0.9946864907980235,
  'f1-score': 0.9924435078681686,
  'support': 2081.0},
 'weighted avg': {'precision': 0.9985672953129137,
  'recall': 0.9985583853916387,
  'f1-score': 0.9985616518860422,
  'support': 2081.0}}

In [14]:
# Create a pypi package with the model
import joblib
joblib.dump(best_model[0], 'false_positive_detection_model.pkl')
joblib.dump(best_model[1], 'false_positive_detection_vectorizer.pkl')

['false_positive_detection_vectorizer.pkl']

In [ ]:
!python -m pip install --upgrade setuptools wheel

In [21]:
import pandas as pd

In [22]:
test = pd.read_csv('../datasets/feature-extraction-paper.csv')
test = test.rename(columns={'copyright': 'content'})
test = test.drop(columns=['falsePositive'])
# to json
test.to_json('test.json', orient='records')

In [23]:
test.head()

,content
0,Copyrights for code taken from ext2:
1,"copyright notice, this list of conditions and ..."
2,"copyright notice, this list of conditions and ..."
3,"Parts derived from drivers/block/rd.c, and dri..."
4,Original copyright notice follows:


In [20]:
data_0

,copyright,falsePositive,pred
0,copyright/agent_tests/Unit/test_copyright src/...,1,1
1,copyright/VERSION-copyright src/spdx2/agent_te...,1,1
2,copyright_list src/cli/fo_folder src/cli/fo_no...,1,1
3,copyright/VERSION-keyword,1,1
4,copyright/VERSION-ecc,1,1
...,...,...,...
19462,copyright_path = '/bin/copyright' keyword_path...,1,1
19463,"copyright violation found"")",1,0
19464,"© 2009 Hewlett-Packard Development Company, L.P.",0,0
19465,copyright_library.py usr/local/lib/python_foss...,1,1


In [28]:
new_df = pd.DataFrame(columns=['copyright', 'falsePositive'])
new_df['copyright'] = X
new_df['falsePositive'] = y
new_df.to_csv('false_positive_detection_dataset.csv', index=False)

In [29]:
test = pd.read_csv('false_positive_detection_dataset.csv')
test.head()

,copyright,falsePositive
0,copyright/agent_tests/Unit/test_copyright src/...,1
1,copyright/VERSION-copyright src/spdx2/agent_te...,1
2,copyright_list src/cli/fo_folder src/cli/fo_no...,1
3,copyright/VERSION-keyword,1
4,copyright/VERSION-ecc,1
